In [99]:
# Modules
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import chromedriver_binary  # Adds chromedriver binary to path
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import time
from datetime import date, datetime
from bs4 import BeautifulSoup
import pandas as pd

In [108]:


class Scraper:
    
    def __init__(self):
        pass
        
    def waitforxpath(self,driver,path,return_element=False,click_element=False): 
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,path)))
        finally:
            s=driver.find_element(By.XPATH, path)
            
        if click_element:
            driver.execute_script("arguments[0].click();", s)
            return driver
            
        if return_element:
            return element
        
        return s
    
    def waitforname(self,driver,tag): 
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME,tag)))
        except:
            return []
            
        s=driver.find_elements(By.TAG_NAME, tag)
        time.sleep(0.5)
        return s
    
    def hide_ads(self,driver):
        all_iframes = self.waitforname(driver,"iframe")
        if len(all_iframes) > 0:
            print("Ad Found\n")
            driver.execute_script("""
                var elems = document.getElementsByTagName("iframe"); 
                for(var i = 0, max = elems.length; i < max; i++)
                     {
                         elems[i].hidden=true;
                     }
                                  """)
            print('Total Ads: ' + str(len(all_iframes)))
        else:
            print('No frames found')
            
        return driver
        
        
    def open_webpage(self,URL = None):
        chrome_options = Options()
        chrome_options.add_argument("--disable-popup-blocking")
        
        


#         chrome_options.add_argument("auto-open-devtools-for-tabs")
#         chrome_options.add_argument("--headless")  
        driver = webdriver.Chrome(options=chrome_options)
        # Set the user agent to a common browser to avoid detection
        # as a bot. You can find a list of common user agents at
        # https://developers.whatismybrowser.com/useragents/explore/
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
        driver.execute_cdp_cmd("Network.setUserAgentOverride", {"userAgent": user_agent})
    
    
        if URL != None:
            print('opening URL- ',URL)
            driver.get(URL)            
        else:
            driver.get('https://fullmatchesreplays.com/highlights/category/premier-league/')
        return driver
    
    def click_body(self,driver):
        actions = ActionChains(driver)
        actions.move_to_element_with_offset(self.waitforname(driver,'body')[0], 0,0)
        actions.move_by_offset(200,200).click().perform()
        return driver
            
    def ad_workaround(self,driver):
        
        ActionChains(driver)\
        .key_down(Keys.TAB)\
        .key_down(Keys.TAB)\
        .key_down(Keys.TAB)\
        .key_down(Keys.TAB)\
        .key_down(Keys.TAB)\
        .key_down(Keys.TAB)\
        .key_down(Keys.TAB)\
        .key_down(Keys.TAB)\
        .key_down(Keys.TAB)\
        .key_down(Keys.ENTER)\
        .perform()
        
        time.sleep(5)
        driver.back()
        return driver

    def download_1st_half(self, driver):
        self.hide_ads(driver)
        self.waitforxpath(driver,"//*[contains(text(),'1st')][1]",click_element=True)
        print(1)
        driver.switch_to.window(driver.window_handles[-1])
        print(2)
        self.hide_ads(driver)
        print(3)
        
        
        time.sleep(11)
        print(5)
        
        for attempt in range(2):
            self.hide_ads(driver)
            try:
                self.waitforxpath(driver,"//*[@class='btn btn-primary btn-captcha']",click_element=True)
            except:
                print('get link failed')
            try:
#                 self.waitforxpath(driver,"//*[@class='main-footer']",click_element=True)
                self.ad_workaround(driver)
            except:
                print('ad workaround failed')  
#             try:
#                 self.waitforxpath(driver,"//*[text()='Fermer']",click_element=True)
#             except:
#                 print('Fermer Ad was not closed')
        # Try this
#         driver.find_element(By.ID, "g-recaptcha-button-submit").click()
        print(7)
        self.ad_workaround(driver)
        print(8)
        self.waitforxpath(driver,"//*[@class='btn btn-primary btn-captcha']",click_element=True)
        print(9)
        self.waitforxpath(driver,"//*[@class='btn btn-success btn-lg get-link']",click_element=True)
        print(10)
        self.hide_ads(driver)
        self.waitforxpath(driver,"//*[text()='Download']",click_element=True)
        self.hide_ads(driver)
        self.waitforxpath(driver,"//*[contains(text(),'360p')]",click_element=True)
        
        return driver
    
    def click_element(self,driver,element):
        driver.execute_script("arguments[0].click();", element)
        return driver
    
    def scrape_match(self,url):
        driver = self.open_webpage(url)
        self.download_1st_half(driver)
        
        #force stop
        self.waitforxpath(driver,error)
        


In [109]:
video_scraper = Scraper()
video_scraper.scrape_match('https://fullmatchesreplays.com/highlights/brighton-hove-albion-vs-aston-villa-13-nov-2022/')

opening URL-  https://fullmatchesreplays.com/highlights/brighton-hove-albion-vs-aston-villa-13-nov-2022/
Ad Found

Total Ads: 8
1
2
Ad Found

Total Ads: 9
3
5
Ad Found

Total Ads: 10
Ad Found

Total Ads: 10
7
8
9


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class='btn btn-success btn-lg get-link']"}
  (Session info: chrome=108.0.5359.98)
Stacktrace:
0   chromedriver                        0x0000000104996a88 chromedriver + 4123272
1   chromedriver                        0x0000000104922778 chromedriver + 3647352
2   chromedriver                        0x00000001045e4ac4 chromedriver + 248516
3   chromedriver                        0x000000010461af68 chromedriver + 470888
4   chromedriver                        0x000000010464d020 chromedriver + 675872
5   chromedriver                        0x000000010460eb10 chromedriver + 420624
6   chromedriver                        0x000000010460fc30 chromedriver + 425008
7   chromedriver                        0x0000000104968ae4 chromedriver + 3934948
8   chromedriver                        0x000000010496bf24 chromedriver + 3948324
9   chromedriver                        0x000000010496c508 chr

In [37]:
dir(By)

['CLASS_NAME',
 'CSS_SELECTOR',
 'ID',
 'LINK_TEXT',
 'NAME',
 'PARTIAL_LINK_TEXT',
 'TAG_NAME',
 'XPATH',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']